In [52]:
import numpy as np
import matplotlib.pyplot as plt
import torch

In [53]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [54]:
data = unpickle('data_batch_1')
X = data[b'data']
Y = data[b'labels']
X.shape

(10000, 3072)

In [55]:
tmp = X[0:5]
tmp = np.reshape(tmp, (-1, 3, 32, 32)  )
tmp = tmp.transpose( (0, 2, 3, 1) )

im1 = tmp[0]
plt.imsave('im1.jpg', im1)

im1 = tmp[1]
plt.imsave('im2.jpg', im1)

im1 = tmp[2]
plt.imsave('im3.jpg', im1)

im1 = tmp[3]
plt.imsave('im4.jpg', im1)

im1 = tmp[4]
plt.imsave('im5.jpg', im1)

In [56]:
X = np.reshape(X, (-1, 3, 32, 32)  )
X = X.astype(np.float32)
#X = X / 255
#X = X.transpose( (0, 3, 1, 2) )
X.shape

(10000, 3, 32, 32)

In [57]:
from torch.utils.data import DataLoader

class my_dataset(torch.utils.data.Dataset):
    def __init__(self, ):
        pass

    def __getitem__(self, index):
        return X[index], Y[index]

    def __len__(self):
        return len(X)
    
ds = my_dataset()

In [58]:
import torch

from torch.utils.data import random_split
from torch.utils.data import DataLoader

train, test = random_split( ds , (  9000   , 1000  ) )
data_loader = torch.utils.data.DataLoader(dataset=train,batch_size=50, shuffle=True)
data_loader2 = torch.utils.data.DataLoader(dataset=test,batch_size=1000, shuffle=True)


In [59]:
import torch.nn as nn
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        
        self.L1 = nn.Conv2d(3, 64, 7, 2, 3)
        self.B1 = nn.BatchNorm2d(64)

        self.L2 = nn.Conv2d(64, 64, 3, 1, 1)
        self.B2 = nn.BatchNorm2d(64)

        self.L3 = nn.Conv2d(64, 64, 3, 1, 1)
        self.B3 = nn.BatchNorm2d(64)

        self.L4 = nn.Conv2d(64, 128, 3, 2, 1)
        self.B4 = nn.BatchNorm2d(128)

        self.L5 = nn.Conv2d(128, 128, 3, 1, 1)
        self.B5 = nn.BatchNorm2d(128)

        self.L6 = nn.Conv2d(64, 128, 1, 2, 0)
        self.B6 = nn.BatchNorm2d(128)

        #self.pool = nn.AdaptiveAvgPool2d(512)

        self.fc = nn.Linear(8192, 10)

    def forward(self, x):
        g1 = self.L1(x)
        g1 = self.B1(g1)
        f1 = torch.relu(g1)

        t = f1

        g2 = self.L2(f1)
        g2 = self.B2(g2)
        f2 = torch.relu(g2)

        g3 = self.L3(f2)
        g3 = self.B3(g3)
        f3 = torch.relu(g3)

        f3 = t + f3

        t2 = f3

        g4 = self.L4(f3)
        g4 = self.B4(g4)
        f4 = torch.relu(g4)
        
        g5 = self.L5(f4)
        g5 = self.B5(g5)
        f5 = torch.relu(g5)

        g6 = self.L6(t2)
        g6 = self.B6(g6)
        f6 = torch.relu(g6)

        f5 = f5 + f6

        #f6 = self.pool(f5)
        f6 = torch.flatten(f5, start_dim = 1)

        g = self.fc(f6)

        return g
    
model = MyNet()
#model = torch.load('model1.pth')

In [60]:
import torchmetrics

lossf = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
metricsf = torchmetrics.Accuracy(task='multiclass', num_classes=10) 

for i in range(15):
    for batchX, batchY in data_loader:
        score = model(batchX)
        score = torch.squeeze(score)
        loss = lossf(score, batchY)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        mse = metricsf(score, batchY)
        print(i, loss, mse)

    print(i, loss, metricsf.compute())
    metricsf.reset()

torch.save(model, 'model1.pth')

0 tensor(2.4595, grad_fn=<NllLossBackward0>) tensor(0.0800)
0 tensor(58.1854, grad_fn=<NllLossBackward0>) tensor(0.1400)
0 tensor(73.5317, grad_fn=<NllLossBackward0>) tensor(0.1400)
0 tensor(73.8006, grad_fn=<NllLossBackward0>) tensor(0.1800)
0 tensor(73.3097, grad_fn=<NllLossBackward0>) tensor(0.2000)
0 tensor(55.1450, grad_fn=<NllLossBackward0>) tensor(0.1600)
0 tensor(55.0021, grad_fn=<NllLossBackward0>) tensor(0.0400)
0 tensor(40.4557, grad_fn=<NllLossBackward0>) tensor(0.1000)
0 tensor(30.9724, grad_fn=<NllLossBackward0>) tensor(0.1200)
0 tensor(15.3539, grad_fn=<NllLossBackward0>) tensor(0.1000)
0 tensor(6.6945, grad_fn=<NllLossBackward0>) tensor(0.0600)
0 tensor(11.3974, grad_fn=<NllLossBackward0>) tensor(0.1800)
0 tensor(5.2402, grad_fn=<NllLossBackward0>) tensor(0.1400)
0 tensor(4.0769, grad_fn=<NllLossBackward0>) tensor(0.1600)
0 tensor(4.9189, grad_fn=<NllLossBackward0>) tensor(0.1400)
0 tensor(5.7620, grad_fn=<NllLossBackward0>) tensor(0.0400)
0 tensor(6.1561, grad_fn=<NllL

In [61]:
model.eval()

for batchX, batchY in data_loader2:
    score = model(batchX)
    score = torch.squeeze(score)
    mse = metricsf(score, batchY)

print( metricsf.compute() )

tensor(0.5070)
